In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import pylab as plt
from fbprophet import Prophet
%matplotlib inline

import scipy.optimize

In [ ]:
df = pd.read_csv('../input/novel-corona-virus-2019-dataset/covid_19_data.csv',parse_dates=['Last Update'], infer_datetime_format=True)
df.rename(columns={'ObservationDate':'Date', 'Country/Region':'Country'}, inplace=True)

In [ ]:
import matplotlib.dates as mdates
import datetime 

sns.set_context('talk')

months = mdates.MonthLocator()
months_fmt = mdates.DateFormatter('%b-%e')

plt.figure(figsize=(8,6))

df_it=df[df.Country=='Italy']
df_ch=df[df.Country=='Switzerland']
t_ch=pd.DatetimeIndex(df_ch.Date)

df_k=df[df.Country=='South Korea']
t_k=pd.to_datetime(pd.DatetimeIndex(df_k.Date))+pd.DateOffset(days=16)

df_hc=df[df['Province/State']=='Hubei']
t_hc=pd.DatetimeIndex(df_hc.Date)+pd.DateOffset(days=36+16)
plt.plot( t_hc,df_hc.Deaths,label='Hubei +52days');

t_it=pd.to_datetime(df_it.Date)+pd.DateOffset(days=16)
plt.plot(t_it,df_it.Deaths,label='Italy +16days');

plt.plot(pd.to_datetime(t_k),df_k.Deaths,label='Korea +16days');

df_fr=df[df.Country=='France']
t_fr=pd.to_datetime(df_fr.Date)+pd.DateOffset(days=5)
plt.plot(t_fr,df_fr.Deaths,'d',label='France +5days', alpha=0.5);

df_uk=df[df.Country=='UK']
t_uk=pd.to_datetime(df_uk.Date)+pd.DateOffset(days=0)
plt.plot(t_uk,df_uk.Deaths,'o',label='UK +0days', alpha=0.5);

df_de=df[df.Country=='Germany']
t_de=pd.to_datetime(df_de.Date)-pd.DateOffset(days=1)
plt.plot(t_de,df_de.Deaths,label='Germany +0days', lw=3);

df_us=df[df.Country=='US']
df_usa=df_us.groupby(by='Date').sum()
t_usa=pd.to_datetime(df_usa.index)+pd.DateOffset(days=7)
plt.plot(t_usa,df_usa.Deaths, '^',label='USA +7days', alpha=0.5);

plt.plot(pd.to_datetime(t_ch),df_ch.Deaths,'o',color='red',label='Switzerland +0days',lw=3);

plt.gca().xaxis.set_major_locator(plt.MaxNLocator(4))
plt.gca().xaxis.set_major_formatter(months_fmt)
plt.xticks(rotation=45, fontsize=12)
plt.gca().grid(True)
plt.legend(loc=4);
plt.yscale('Log');
date1 = datetime.date(2020, 3, 1)
date2 = datetime.date(2020, 5, 12)

plt.axis([date1, date2,1,10000]);
plt.ylabel('Deaths');

In [ ]:
def sigmoid(p,x): 
    x0,y0,c,k=p 
    y = c / (1 + np.exp(-k*(x-x0))) + y0 
    return y 

def residuals(p,x,y):
    return y - sigmoid(p,x)

def date2int(date):
    x=pd.to_datetime( date).copy()
    x=x.reset_index()
    x=x.index.values
    return x

In [ ]:
x=date2int( df_it.Date) +16
y=df_it.Deaths.values

In [ ]:

p_guess=np.array([34.,0,8000,0.2])
p_it, cov, infodict, mesg, ier = scipy.optimize.leastsq(
    residuals,p_guess,args=(x,y),full_output=1,maxfev=2000)  

In [ ]:
def fit_sig(df_hc, tdelta=0, p_guess=np.array([34.,0,8000,0.2])):

#     x=date2int(df_hc.Date)+ tdelta
    x=(pd.to_datetime(df_hc.Date)-pd.to_datetime('01/22/2020')).dt.days + tdelta
    y=df_hc.Deaths
    
    p_it, cov, infodict, mesg, ier = scipy.optimize.leastsq(
    residuals,p_guess,args=(x,y),full_output=1,maxfev=2000)  
    return p_it,x

In [ ]:
p_c,x_c= fit_sig(df_hc)

In [ ]:
plt.figure(figsize=(8,6))

x2=np.arange(0,100)


it_dead=sigmoid(p_it,x2)
plt.plot(x2,it_dead,'-', label='Italy forecast');
plt.plot(x,y,'o',label='Italy');

it_icu=1*np.diff(it_dead)
it_icu_total=np.convolve(it_icu,np.ones((6,))/1.)
x3=np.arange(0,it_icu_total.shape[0])
plt.plot(x2[1:],it_icu,'-', label='Italy ICU daily demand');
plt.plot(x3 ,it_icu_total,'-', label='Italy ICU total demand');


plt.plot(x2,sigmoid(p_c,x2),'-', label='Hubei forecast');
plt.plot(date2int(df_hc.Date),df_hc.Deaths,'o',label='Hubei +52days');

plt.gca().grid(True)
plt.legend(loc=2, fontsize=10);
plt.ylabel('Deaths');

In [ ]:
plt.figure(figsize=(8,6))

# df_it=df[df.Country=='Italy']
# df_ch=df[df.Country=='Switzerland']
# df_hc=df[df['Province/State']=='Hubei']

x2=np.arange(0,160)

p_it,x_it= fit_sig(df_it,tdelta=18)
plt.plot(x2 ,sigmoid(p_it,x2),'-',color='steelblue', label='Italy forecast',alpha=0.5);
plt.plot(x_it,df_it.Deaths,'o',color='steelblue',label='Italy +18days',alpha=0.5);


p_c,x_c= fit_sig(df_hc,tdelta=54,p_guess=np.array([80.,0,8000,0.2]))
plt.plot(x2,sigmoid(p_c,x2),'-',color='darkred', label='Hubei forecast',alpha=0.5);
plt.plot(x_c,df_hc.Deaths,'o',color='darkred',label='Hubei +53days',alpha=0.5);

p_ch,x_ch= fit_sig(df_ch,tdelta=0)
#plt.plot(x2,sigmoid(p_ch,x2),'-',color='darkgreen', label='CH forecast');
plt.plot(x_ch,df_ch.Deaths,'o',color='darkgreen',label='CH +0days');

plt.gca().grid(True)
plt.legend(loc=4, fontsize=10);
# plt.axis([40,100,0,15000]);
# plt.yscale('Log');
plt.ylabel('Deaths');


In [ ]:
 plt.figure(figsize=(8,6))

# df_it=df[df.Country=='Italy']
# df_ch=df[df.Country=='Switzerland']
# df_hc=df[df['Province/State']=='Hubei']

x2=np.arange(0,100)

# p_it,x_it= fit_sig(df_it,tdelta=18)
plt.plot(x2 ,sigmoid(p_it,x2),'-',color='steelblue', label='Italy forecast',alpha=0.5);
plt.plot(x_it,df_it.Deaths,'o',color='steelblue',label='Italy +18days',alpha=0.5);


# p_c,x_c= fit_sig(df_hc,tdelta=54,p_guess=np.array([80.,0,8000,0.2]))
plt.plot(x2,sigmoid(p_c,x2),'-',color='darkred', label='Hubei forecast',alpha=0.5);
plt.plot(x_c,df_hc.Deaths,'o',color='darkred',label='Hubei +53days',alpha=0.5);

# p_ch,x_ch= fit_sig(df_ch,tdelta=0)
plt.plot(x2,sigmoid(p_ch,x2),'-',color='darkgreen', label='CH forecast');
plt.plot(x_ch,df_ch.Deaths,'o',color='darkgreen',label='CH +0days');

plt.gca().grid(True)
plt.legend(loc=4, fontsize=10);
plt.axis([40,100,0,500]);
# plt.yscale('Log');
plt.ylabel('Deaths');

